# Solar Panel Project

## Setup Project
Here are the datasets that are provided.

In [123]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

data_path = "./data"

daylight = pd.read_json(f'{data_path}/Daylight.json')
houses = pd.read_json(f'{data_path}/Houses.json')
installers = pd.read_json(f'{data_path}/Installers.json')
panels = pd.read_json(f'{data_path}/Panels.json')
tariffs = pd.read_json(f'{data_path}/Tariffs.json')

## Panel Fitter
Fit panels onto house using roof dimensions.

In [124]:
from sklearn.base import BaseEstimator, TransformerMixin
import math

class PanelFitter(BaseEstimator, TransformerMixin):
    def __init__(self, panels: pd.DataFrame):
        panels = panels.copy()
        panels["Length"] = panels["Size"].apply(lambda x: self._extract_dimension(x, 0))
        panels["Width"] = panels["Size"].apply(lambda x: self._extract_dimension(x, 1))
        panels = panels.drop(['Size', 'Url'], axis=1)
        self.panels = panels
    
    def _extract_dimension(self, x: str, dim: int):
        return float(x.split(",")[dim])

    def _fit_panels(self, panel, max_length: float, max_width: float):
        lengthCount = math.floor(max_length / panel["Length"])
        widthCount = math.floor((max_width) / panel["Width"])
        panel["Count"] = lengthCount * widthCount

        if panel["Count"] == 0:
            return panel
            
        panel["TotalCost"] = panel["Count"] * panel["Cost"]
        panel["TotalWeight"] = panel["Count"] * panel["Weight"]
        panel["TotalPower"] = panel["Count"] * panel["Power"]
        panel["UsefulPower"] = (panel["Efficiency"] / 100) * panel["TotalPower"]
        return panel

    def fit(self, X, y=None):
        return self
    
    def transform(self, roof_size: tuple[float, float]):
        panels = self.panels.copy()
        max_length, max_width = roof_size
        panels = panels.apply(lambda x: self._fit_panels(x, max_length, max_width), axis=1)
        panels.drop(["Length", "Width", "Power", "Weight", "Cost"], axis=1, inplace=True)
        return panels

## Algorithms
Use fitted panel data to find optimal set of panels.

In [207]:
def get_most_efficient_panels(panels: pd.DataFrame, roof_size: tuple[float, float]):
    fitted_panels = PanelFitter(panels).transform(roof_size)
    return fitted_panels.sort_values(by="UsefulPower", ascending=False)

def get_least_cost_panels(panels: pd.DataFrame, roof_size: tuple[float, float]):
    fitted_panels = PanelFitter(panels).transform(roof_size)
    return fitted_panels.sort_values(by="TotalCost", ascending=True)

In [208]:
def get_best_options(panels: pd.DataFrame, budget: float, roof_size: tuple[float, float]):
    fitted_panels = PanelFitter(panels).transform(roof_size)

    if "TotalCost" not in fitted_panels.columns:
        return Exception("No panels found for this roof size")

    within_budget = fitted_panels.where(fitted_panels["TotalCost"] <= budget).dropna()

    if within_budget.shape[0] == 0:
        return Exception("No panels found within budget")

    return within_budget.sort_values(by="UsefulPower", ascending=False)

budget = 1000
roof_size = (10, 3)

options = get_best_options(panels, budget, roof_size)
least_cost = get_least_cost_panels(panels, (1.5, 10))

least_cost

,Count,Efficiency,Manufacturer,Model,TotalCost,TotalPower,TotalWeight,UsefulPower
0,6,20.2,Solaria,PowerXT® Pure Black™400W,2004.18,2400.0,25.8,484.800
1,19,21.8,Solar Power Supply,120W SHINGLE,2469.05,2242.0,133.0,488.756
2,0,18.7,Trina,325W Trina Honey Mono All Black,NaN,NaN,NaN,NaN
3,0,17.9,Viridian,Viridian Clearline 280W Poly Black,NaN,NaN,NaN,NaN
4,0,20.3,Q Cells,385W Q Cells ML Mono Q Peak Duo G9+,NaN,NaN,NaN,NaN
5,0,21.1,LG,375W LG Mono Neon R V5,NaN,NaN,NaN,NaN
